# Generative Adversarial Networks for Data Augmentation | Part 2
- In this second part, we will be using the augmented datasets from part 1 to perform the classification task.

# 1. Importing the Packages & Boilerplate Code

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from shutil import copyfile
from tabulate import tabulate
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import tensorflow.keras.layers as tfl

In [2]:
# Setting the seeds
SEED = 0
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# Making sure that Tensorflow is able to detect the GPU
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# 2. Importing the Train/Test Sets

In [4]:
# Importing the Test Dataset
print("For Test Dataset:")
df_test = pd.read_csv("../input/cifar10/test_x.csv")
y_test = pd.read_csv("../input/cifar10/test_y.csv")
df_test = np.array(df_test)
y_test = np.array(y_test)
print(df_test.shape, y_test.shape)

# Reshaping the dataset
df_test = np.reshape(df_test, (-1, 3, 32, 32))
print(df_test.shape)

# Reshaping, rescaling and one-hot encoding
df_test = np.transpose(np.array(df_test), (0, 2, 3, 1))
df_test = df_test / 255
y_test_oh = tf.one_hot(np.ravel(y_test), depth = 10)
print(df_test.shape, y_test_oh.shape)

# =========================================================
print("For Train Dataset:")
# Importing the Labelled Training Dataset
df_train = pd.read_csv("../input/cifar10/train_lab_x.csv")
y_train = pd.read_csv("../input/cifar10/train_lab_y.csv")
df_train = np.array(df_train)
y_train = np.array(y_train)
print(df_train.shape, y_train.shape)

# Reshaping, rescaling and one-hot encoding
df_train = np.reshape(df_train, (-1, 3, 32, 32))
df_train = np.transpose(np.array(df_train), (0, 2, 3, 1))
df_train = df_train / 255
print(df_train.shape)

For Test Dataset:
(10000, 3072) (10000, 1)
(10000, 3, 32, 32)
(10000, 32, 32, 3) (10000, 10)
For Train Dataset:
(40006, 3072) (40006, 1)
(40006, 32, 32, 3)


# 3. Image Augmentation on 25% of the Training Dataset
## 3.1. Augmenting the Training Dataset

In [5]:
# Importing the Augmented Dataset
df_aug = pd.read_csv("../input/cifar10/df_25per_aug.csv")
y_aug = pd.read_csv("../input/cifar10/y_25per_aug.csv")
df_aug = np.array(df_aug)
y_aug = np.array(y_aug)

# Reshaping, rescaling and one-hot encoding
df_aug = np.reshape(df_aug, (-1, 3, 32, 32))
df_aug = np.transpose(np.array(df_aug), (0, 2, 3, 1))

# Concatenating the Training with Augmenting Dataset
df_aug = np.concatenate([df_train, df_aug], axis=0)
y_aug = np.concatenate([y_train, y_aug], axis=0)
print(df_aug.shape, y_aug.shape)

# Creating a random permutation & shuffling the dataset
perm = np.random.permutation(df_aug.shape[0])
df_aug = np.array(df_aug[perm, : , : , : ])
y_aug = y_aug[perm]
y_aug_oh = tf.one_hot(np.ravel(y_aug), depth = 10)
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

(50022, 32, 32, 3) (50022, 1)
(50022, 32, 32, 3) (50022, 1) (50022, 10)


## 3.2. Training the Baseline Model on the Augmented Dataset

In [6]:
# Importing the Baseline Model Architecture
copyfile(src = "../input/dcai-rw/baseline_arch.py", dst = "../working/baseline_arch.py")
from baseline_arch import cnn_model

# Creating Batches from the Augmented Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((df_aug, y_aug_oh)).batch(32)

In [7]:
num_epochs = [10, 20, 30, 40, 50]
train_loss, test_loss, train_acc, test_acc = [], [], [], []

for epochs in num_epochs:
    # Training the Model
    conv_model = cnn_model((32, 32, 3))
    conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    conv_model.fit(train_dataset, epochs = epochs)
    
    # Predicting on the Train/Test Datasets
    preds_train = conv_model.predict(df_aug)
    preds_test = conv_model.predict(df_test)

    # Finding the Predicted Classes
    cls_train = np.argmax(preds_train, axis = 1)
    cls_test = np.argmax(preds_test, axis = 1)
    
    # Finding the Train/Test set Loss
    train_loss.append(log_loss(y_aug_oh, preds_train))
    test_loss.append(log_loss(y_test_oh, preds_test))
    train_acc.append(accuracy_score(y_aug, cls_train))
    test_acc.append(accuracy_score(y_test, cls_test))
    
    print("For ", epochs, " Epochs:")
    print("Log-loss for Train Dataset = ", train_loss[-1])
    print("Log-loss for Test Dataset = ", test_loss[-1])
    print("Accuracy for Train Dataset = ", train_acc[-1])
    print("Accuracy for Test Dataset = ", test_acc[-1])
    print()

Epoch 1/10
1564/1564 [==============================] - 14s 5ms/step - loss: 1.6657 - accuracy: 0.3963
Epoch 2/10
1564/1564 [==============================] - 7s 5ms/step - loss: 1.2209 - accuracy: 0.5707
Epoch 3/10
1564/1564 [==============================] - 8s 5ms/step - loss: 1.0402 - accuracy: 0.6388
Epoch 4/10
1564/1564 [==============================] - 7s 5ms/step - loss: 0.9371 - accuracy: 0.6776
Epoch 5/10
1564/1564 [==============================] - 8s 5ms/step - loss: 0.8650 - accuracy: 0.7019
Epoch 6/10
1564/1564 [==============================] - 8s 5ms/step - loss: 0.8095 - accuracy: 0.7207
Epoch 7/10
1564/1564 [==============================] - 8s 5ms/step - loss: 0.7726 - accuracy: 0.7341
Epoch 8/10
1564/1564 [==============================] - 7s 5ms/step - loss: 0.7302 - accuracy: 0.7452
Epoch 9/10
1564/1564 [==============================] - 7s 5ms/step - loss: 0.6882 - accuracy: 0.7601
Epoch 10/10
1564/1564 [==============================] - 8s 5ms/step - loss: 0.66

In [8]:
# Training the Model with the best hyper-parameter settings
ind = np.argmax(test_acc)
best_num_epochs = num_epochs[ind]
conv_model = cnn_model((32, 32, 3))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
conv_model.fit(train_dataset, epochs = best_num_epochs)

# Saving the model along with it's weights
conv_model.save('gan_augmented_all.h5')

Epoch 1/50
1564/1564 [==============================] - 8s 5ms/step - loss: 1.6535 - accuracy: 0.4015
Epoch 2/50
1564/1564 [==============================] - 7s 5ms/step - loss: 1.2096 - accuracy: 0.5761
Epoch 3/50
1564/1564 [==============================] - 7s 5ms/step - loss: 1.0381 - accuracy: 0.6401
Epoch 4/50
1564/1564 [==============================] - 7s 5ms/step - loss: 0.9353 - accuracy: 0.6761
Epoch 5/50
1564/1564 [==============================] - 7s 5ms/step - loss: 0.8558 - accuracy: 0.7029
Epoch 6/50
1564/1564 [==============================] - 7s 5ms/step - loss: 0.7996 - accuracy: 0.7210
Epoch 7/50
1564/1564 [==============================] - 10s 7ms/step - loss: 0.7609 - accuracy: 0.7380
Epoch 8/50
1564/1564 [==============================] - 7s 5ms/step - loss: 0.7195 - accuracy: 0.7524
Epoch 9/50
1564/1564 [==============================] - 8s 5ms/step - loss: 0.6869 - accuracy: 0.7629
Epoch 10/50
1564/1564 [==============================] - 7s 5ms/step - loss: 0.65

## 3.3. Predicting the Performance

In [9]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_aug)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Train/Test set Loss
print("Log-loss for Augmented Dataset = ", log_loss(y_aug_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Augmented Dataset = ", accuracy_score(y_aug, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

Log-loss for Augmented Dataset =  0.10731828685173772
Log-loss for Test Dataset =  0.8453338678017722
Accuracy for Augmented Dataset =  0.9678741353804327
Accuracy for Test Dataset =  0.7547


# 4. Image Augmentation for Class Balancing
## 4.1. Augmenting the Training Dataset

In [10]:
# Importing the Augmented Dataset
df_aug = pd.read_csv("../input/cifar10/df_clsbal_aug.csv")
y_aug = pd.read_csv("../input/cifar10/y_clsbal_aug.csv")
df_aug = np.array(df_aug)
y_aug = np.array(y_aug)

# Reshaping, rescaling and one-hot encoding
df_aug = np.reshape(df_aug, (-1, 3, 32, 32))
df_aug = np.transpose(np.array(df_aug), (0, 2, 3, 1))

# Concatenating the Training with Augmenting Dataset
df_aug = np.concatenate([df_train, df_aug], axis=0)
y_aug = np.concatenate([y_train, y_aug], axis=0)
print(df_aug.shape, y_aug.shape)

# Creating a random permutation & shuffling the dataset
perm = np.random.permutation(df_aug.shape[0])
df_aug = np.array(df_aug[perm, : , : , : ])
y_aug = y_aug[perm]
y_aug_oh = tf.one_hot(np.ravel(y_aug), depth = 10)
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

(43780, 32, 32, 3) (43780, 1)
(43780, 32, 32, 3) (43780, 1) (43780, 10)


## 4.2. Training the Baseline Model on the Augmented Dataset

In [11]:
# Importing the Baseline Model Architecture
copyfile(src = "../input/dcai-rw/baseline_arch.py", dst = "../working/baseline_arch.py")
from baseline_arch import cnn_model

# Creating Batches from the Augmented Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((df_aug, y_aug_oh)).batch(32)

In [12]:
num_epochs = [10, 20, 30, 40, 50]
train_loss, test_loss, train_acc, test_acc = [], [], [], []

for epochs in num_epochs:
    # Training the Model
    conv_model = cnn_model((32, 32, 3))
    conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    conv_model.fit(train_dataset, epochs = epochs)
    
    # Predicting on the Train/Test Datasets
    preds_train = conv_model.predict(df_aug)
    preds_test = conv_model.predict(df_test)

    # Finding the Predicted Classes
    cls_train = np.argmax(preds_train, axis = 1)
    cls_test = np.argmax(preds_test, axis = 1)
    
    # Finding the Train/Test set Loss
    train_loss.append(log_loss(y_aug_oh, preds_train))
    test_loss.append(log_loss(y_test_oh, preds_test))
    train_acc.append(accuracy_score(y_aug, cls_train))
    test_acc.append(accuracy_score(y_test, cls_test))
    
    print("For ", epochs, " Epochs:")
    print("Log-loss for Train Dataset = ", train_loss[-1])
    print("Log-loss for Test Dataset = ", test_loss[-1])
    print("Accuracy for Train Dataset = ", train_acc[-1])
    print("Accuracy for Test Dataset = ", test_acc[-1])
    print()

Epoch 1/10
1369/1369 [==============================] - 7s 5ms/step - loss: 1.7807 - accuracy: 0.3565
Epoch 2/10
1369/1369 [==============================] - 6s 5ms/step - loss: 1.3726 - accuracy: 0.5170
Epoch 3/10
1369/1369 [==============================] - 7s 5ms/step - loss: 1.1597 - accuracy: 0.5973
Epoch 4/10
1369/1369 [==============================] - 7s 5ms/step - loss: 1.0268 - accuracy: 0.6484
Epoch 5/10
1369/1369 [==============================] - 7s 5ms/step - loss: 0.9432 - accuracy: 0.6766
Epoch 6/10
1369/1369 [==============================] - 7s 5ms/step - loss: 0.8757 - accuracy: 0.6994
Epoch 7/10
1369/1369 [==============================] - 7s 5ms/step - loss: 0.8314 - accuracy: 0.7120
Epoch 8/10
1369/1369 [==============================] - 6s 5ms/step - loss: 0.7875 - accuracy: 0.7299
Epoch 9/10
1369/1369 [==============================] - 7s 5ms/step - loss: 0.7450 - accuracy: 0.7450
Epoch 10/10
1369/1369 [==============================] - 7s 5ms/step - loss: 0.714

In [13]:
# Training the Model with the best hyper-parameter settings
ind = np.argmax(test_acc)
best_num_epochs = num_epochs[ind]
conv_model = cnn_model((32, 32, 3))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
conv_model.fit(train_dataset, epochs = best_num_epochs)

# Saving the model along with it's weights
conv_model.save('gan_augmented_cls_imbalance.h5')

Epoch 1/30
1369/1369 [==============================] - 7s 5ms/step - loss: 1.7731 - accuracy: 0.3570
Epoch 2/30
1369/1369 [==============================] - 6s 5ms/step - loss: 1.3341 - accuracy: 0.5298
Epoch 3/30
1369/1369 [==============================] - 6s 5ms/step - loss: 1.1515 - accuracy: 0.5989
Epoch 4/30
1369/1369 [==============================] - 7s 5ms/step - loss: 1.0429 - accuracy: 0.6404
Epoch 5/30
1369/1369 [==============================] - 7s 5ms/step - loss: 0.9531 - accuracy: 0.6697
Epoch 6/30
1369/1369 [==============================] - 7s 5ms/step - loss: 0.9012 - accuracy: 0.6900
Epoch 7/30
1369/1369 [==============================] - 6s 5ms/step - loss: 0.8484 - accuracy: 0.7058
Epoch 8/30
1369/1369 [==============================] - 7s 5ms/step - loss: 0.8069 - accuracy: 0.7217
Epoch 9/30
1369/1369 [==============================] - 7s 5ms/step - loss: 0.7684 - accuracy: 0.7359
Epoch 10/30
1369/1369 [==============================] - 7s 5ms/step - loss: 0.737

## 4.3. Predicting the Performance

In [14]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_aug)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Train/Test set Loss
print("Log-loss for Augmented Dataset = ", log_loss(y_aug_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Augmented Dataset = ", accuracy_score(y_aug, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

Log-loss for Augmented Dataset =  0.21893769497461882
Log-loss for Test Dataset =  0.7985118761176211
Accuracy for Augmented Dataset =  0.9294883508451348
Accuracy for Test Dataset =  0.7533


# 5. Image Augmentation based on Class-wise Performance
## 5.1. Augmenting the Training Dataset

In [15]:
# Importing the Augmented Dataset
df_aug = pd.read_csv("../input/cifar10/df_clsper_aug.csv")
y_aug = pd.read_csv("../input/cifar10/y_clsper_aug.csv")
df_aug = np.array(df_aug)
y_aug = np.array(y_aug)

# Reshaping, rescaling and one-hot encoding
df_aug = np.reshape(df_aug, (-1, 3, 32, 32))
df_aug = np.transpose(np.array(df_aug), (0, 2, 3, 1))

# Concatenating the Training with Augmenting Dataset
df_aug = np.concatenate([df_train, df_aug], axis=0)
y_aug = np.concatenate([y_train, y_aug], axis=0)
print(df_aug.shape, y_aug.shape)

# Creating a random permutation & shuffling the dataset
perm = np.random.permutation(df_aug.shape[0])
df_aug = np.array(df_aug[perm, : , : , : ])
y_aug = y_aug[perm]
y_aug_oh = tf.one_hot(np.ravel(y_aug), depth = 10)
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

(50054, 32, 32, 3) (50054, 1)
(50054, 32, 32, 3) (50054, 1) (50054, 10)


## 5.2. Training the Baseline Model on the Augmented Dataset

In [16]:
# Importing the Baseline Model Architecture
copyfile(src = "../input/dcai-rw/baseline_arch.py", dst = "../working/baseline_arch.py")
from baseline_arch import cnn_model

# Creating Batches from the Augmented Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((df_aug, y_aug_oh)).batch(32)

In [17]:
num_epochs = [10, 20, 30, 40, 50]
train_loss, test_loss, train_acc, test_acc = [], [], [], []

for epochs in num_epochs:
    # Training the Model
    conv_model = cnn_model((32, 32, 3))
    conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    conv_model.fit(train_dataset, epochs = epochs)
    
    # Predicting on the Train/Test Datasets
    preds_train = conv_model.predict(df_aug)
    preds_test = conv_model.predict(df_test)

    # Finding the Predicted Classes
    cls_train = np.argmax(preds_train, axis = 1)
    cls_test = np.argmax(preds_test, axis = 1)
    
    # Finding the Train/Test set Loss
    train_loss.append(log_loss(y_aug_oh, preds_train))
    test_loss.append(log_loss(y_test_oh, preds_test))
    train_acc.append(accuracy_score(y_aug, cls_train))
    test_acc.append(accuracy_score(y_test, cls_test))
    
    print("For ", epochs, " Epochs:")
    print("Log-loss for Train Dataset = ", train_loss[-1])
    print("Log-loss for Test Dataset = ", test_loss[-1])
    print("Accuracy for Train Dataset = ", train_acc[-1])
    print("Accuracy for Test Dataset = ", test_acc[-1])
    print()

Epoch 1/10
1565/1565 [==============================] - 8s 5ms/step - loss: 1.6501 - accuracy: 0.3918
Epoch 2/10
1565/1565 [==============================] - 7s 5ms/step - loss: 1.2648 - accuracy: 0.5489
Epoch 3/10
1565/1565 [==============================] - 8s 5ms/step - loss: 1.0858 - accuracy: 0.6210
Epoch 4/10
1565/1565 [==============================] - 7s 5ms/step - loss: 0.9735 - accuracy: 0.6605
Epoch 5/10
1565/1565 [==============================] - 7s 5ms/step - loss: 0.8975 - accuracy: 0.6881
Epoch 6/10
1565/1565 [==============================] - 8s 5ms/step - loss: 0.8346 - accuracy: 0.7093
Epoch 7/10
1565/1565 [==============================] - 7s 5ms/step - loss: 0.7838 - accuracy: 0.7303
Epoch 8/10
1565/1565 [==============================] - 7s 5ms/step - loss: 0.7431 - accuracy: 0.7430
Epoch 9/10
1565/1565 [==============================] - 7s 5ms/step - loss: 0.7105 - accuracy: 0.7553
Epoch 10/10
1565/1565 [==============================] - 8s 5ms/step - loss: 0.681

In [18]:
# Training the Model with the best hyper-parameter settings
ind = np.argmax(test_acc)
best_num_epochs = num_epochs[ind]
conv_model = cnn_model((32, 32, 3))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
conv_model.fit(train_dataset, epochs = best_num_epochs)

# Saving the model along with it's weights
conv_model.save('gan_augmented_cls_performance_wise.h5')

Epoch 1/50
1565/1565 [==============================] - 9s 5ms/step - loss: 1.6843 - accuracy: 0.3771
Epoch 2/50
1565/1565 [==============================] - 8s 5ms/step - loss: 1.2710 - accuracy: 0.5447
Epoch 3/50
1565/1565 [==============================] - 7s 5ms/step - loss: 1.0766 - accuracy: 0.6238
Epoch 4/50
1565/1565 [==============================] - 8s 5ms/step - loss: 0.9473 - accuracy: 0.6742
Epoch 5/50
1565/1565 [==============================] - 7s 5ms/step - loss: 0.8651 - accuracy: 0.7020
Epoch 6/50
1565/1565 [==============================] - 7s 5ms/step - loss: 0.8097 - accuracy: 0.7204
Epoch 7/50
1565/1565 [==============================] - 7s 5ms/step - loss: 0.7571 - accuracy: 0.7402
Epoch 8/50
1565/1565 [==============================] - 8s 5ms/step - loss: 0.7182 - accuracy: 0.7529
Epoch 9/50
1565/1565 [==============================] - 7s 5ms/step - loss: 0.6887 - accuracy: 0.7659
Epoch 10/50
1565/1565 [==============================] - 7s 5ms/step - loss: 0.653

## 5.3. Predicting the Performance

In [19]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_aug)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Train/Test set Loss
print("Log-loss for Augmented Dataset = ", log_loss(y_aug_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Augmented Dataset = ", accuracy_score(y_aug, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

Log-loss for Augmented Dataset =  0.08465463174989617
Log-loss for Test Dataset =  0.8320012305460958
Accuracy for Augmented Dataset =  0.9758061293802693
Accuracy for Test Dataset =  0.7664
